In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train_df = pd.read_csv('input/test.csv')
test_df = pd.read_csv('input/train.csv')
len(test_df)

input\sample_submission.csv
input\test.csv
input\train.csv


8693

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def apply_ml_model(model, X, y, test_size=0.2, random_state=None):
    """
    Apply a machine learning model on the given data and calculate accuracy.

    Parameters:
        model: The machine learning model to be applied.
        X: The features of the dataset.
        y: The target variable of the dataset.
        test_size: The proportion of the dataset to include in the test split.
        random_state: Random seed for reproducibility.

    Returns:
        accuracy: Accuracy of the model on the test set.
    """
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    
    return accuracy


In [3]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Load your data here
# Assuming you have features in X and target variable in y
SEED = 42
# Define Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=SEED)

# Apply Random Forest model
rf_accuracy = apply_ml_model(rf_model, X, y)
print("Random Forest Accuracy:", rf_accuracy)

# Define XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=SEED)

# Apply XGBoost model
xgb_accuracy = apply_ml_model(xgb_model, X, y)
print("XGBoost Accuracy:", xgb_accuracy)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Define Logistic Regression model
logreg_model = LogisticRegression()

# Apply Logistic Regression model
logreg_accuracy = apply_ml_model(logreg_model, X, y)
print("Logistic Regression Accuracy:", logreg_accuracy)

# Define K-Nearest Neighbors model
knn_model = KNeighborsClassifier(n_neighbors=5)

# Apply K-Nearest Neighbors model
knn_accuracy = apply_ml_model(knn_model, X, y)
print("K-Nearest Neighbors Accuracy:", knn_accuracy)


NameError: name 'apply_ml_model' is not defined

# Exploratory Data Analysis

Exploratory data analysis (EDA) is performed on the datasat to investigate features. Firstly the target feature is plot as pie chart to illustrate the distrubtion of passengers that were transported or not.

In [1]:
# Figure size
plt.figure(figsize=(6,6))

# Pie plot
train_data['Transported'].value_counts().plot.pie(autopct='%1.1f%%', shadow=False, textprops={'fontsize':16}).set_title("Target distribution")

NameError: name 'plt' is not defined